In [1]:
import graphlab

A newer version of GraphLab Create (v1.9) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [14]:
import numpy as np

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

##Model with All Features

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [8]:
model_all = graphlab.linear_regression.create(sales, target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10, 
                                              verbose=False)

Which features have a non-zero coeficient?

In [10]:
model_all.get("coefficients").print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen?

1. bathrooms
2. sqft_living
3. sqft_living_sqrt
4. grade
5. sqft_above

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1)
(training, validation) = training_and_validation.random_split(0.5, seed=1)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
def get_residual_sum_of_squares(model, data, outcome):
    predicted_price = model.predict(data)
    residuals = predicted_price - outcome
    RSS = (residuals * residuals).sum()
    return(RSS)  

In [23]:
residuals = []
rss_min = float('inf')
best_model = 0
best_l1 = 0
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, l1_penalty=l1_penalty, 
                                              verbose=False)
    rss = get_residual_sum_of_squares(model, validation, validation['price'])
    residuals.append(rss)
    if rss < rss_min:
        rss_min = rss
        best_model = model
        best_l1 = l1_penalty
    print 'For l1_penalty: ' + str(l1_penalty) + ' the rss was: ' + str(rss) + '\n'
print 'The min rss was ' + str(rss_min) + ' had a l1_penalty of ' + str(best_l1)
    

For l1_penalty: 10.0 the rss was: 6.25766285142e+14

For l1_penalty: 31.6227766017 the rss was: 6.25766285362e+14

For l1_penalty: 100.0 the rss was: 6.25766286058e+14

For l1_penalty: 316.227766017 the rss was: 6.25766288257e+14

For l1_penalty: 1000.0 the rss was: 6.25766295212e+14

For l1_penalty: 3162.27766017 the rss was: 6.25766317206e+14

For l1_penalty: 10000.0 the rss was: 6.25766386761e+14

For l1_penalty: 31622.7766017 the rss was: 6.25766606749e+14

For l1_penalty: 100000.0 the rss was: 6.25767302792e+14

For l1_penalty: 316227.766017 the rss was: 6.25769507644e+14

For l1_penalty: 1000000.0 the rss was: 6.25776517727e+14

For l1_penalty: 3162277.66017 the rss was: 6.25799062845e+14

For l1_penalty: 10000000.0 the rss was: 6.25883719085e+14

The min rss was 6.25766285142e+14 had a l1_penalty of 10.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
10.0
2. What is the RSS on TEST data of the model with the best `l1_penalty`?
156983602381664.2

In [24]:
get_residual_sum_of_squares(best_model, testing, testing['price'])

156983602381664.2

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have? 18

In [26]:
best_model.get("coefficients").print_rows(num_rows=18)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [27]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [29]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [37]:
l1_penalty_to_non_zeros = {}
non_zero_array = []
for l1_penalty in np.logspace(8,10, num=20):
    model = graphlab.linear_regression.create(training, target='price', 
                                          features=all_features,
                                          validation_set=None, 
                                          l2_penalty=0, l1_penalty=l1_penalty, 
                                          verbose=False)
    parameters = model['coefficients']['value']
    non_zeros = parameters.nnz()
    non_zero_array.append(non_zeros)
    print 'For l1 penalty ' + str(l1_penalty) + ' the number of non-zeros were: ' + str(non_zeros) 
    l1_penalty_to_non_zeros[l1_penalty] = non_zeros 

For l1 penalty 100000000.0 the number of non-zeros were: 18
For l1 penalty 127427498.57 the number of non-zeros were: 18
For l1 penalty 162377673.919 the number of non-zeros were: 18
For l1 penalty 206913808.111 the number of non-zeros were: 18
For l1 penalty 263665089.873 the number of non-zeros were: 17
For l1 penalty 335981828.628 the number of non-zeros were: 17
For l1 penalty 428133239.872 the number of non-zeros were: 17
For l1 penalty 545559478.117 the number of non-zeros were: 17
For l1 penalty 695192796.178 the number of non-zeros were: 17
For l1 penalty 885866790.41 the number of non-zeros were: 16
For l1 penalty 1128837891.68 the number of non-zeros were: 15
For l1 penalty 1438449888.29 the number of non-zeros were: 15
For l1 penalty 1832980710.83 the number of non-zeros were: 13
For l1 penalty 2335721469.09 the number of non-zeros were: 12
For l1 penalty 2976351441.63 the number of non-zeros were: 10
For l1 penalty 3792690190.73 the number of non-zeros were: 6
For l1 penalt

In [32]:
non_zero_array

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

For l1 penalty 6158482110.66 the number of non-zeros were: 3<br>For l1 penalty 2976351441.63 the number of non-zeros were: 10

In [38]:
l1_penalty_max = 6158482110.66
l1_penalty_min = 2976351441.63

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [40]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [48]:
residuals = []
rss_min = float('inf')
best_model = 0
best_l1 = 0
best_model_with_7 = 0
min_rss_for_7 = float('inf')
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', 
                                          features=all_features,
                                          validation_set=None, 
                                          l2_penalty=0, l1_penalty=l1_penalty, 
                                          verbose=False)
    rss = get_residual_sum_of_squares(model, validation, validation['price'])
    parameters = model['coefficients']['value']
    non_zeros = parameters.nnz()
    print 'For l1 value of : ' + str(l1_penalty) + ' the non-zeros were: ' + str(non_zeros)
    print 'And rss value of : ' + str(rss)  + '\n'
    if rss < rss_min:
        rss_min = rss
        best_model = model
        best_l1 = l1_penalty
    if non_zeros == max_nonzeros:
        best_model_with_7 = model
        min_rss_for_7 = rss
print 'The min rss was: ' + str(rss_min) + '\n'
print 'The min rss for sparsity of 7 was: ' + str(min_rss_for_7)

For l1 value of : 2976351441.63 the non-zeros were: 10
And rss value of : 9.66925692362e+14

For l1 value of : 3143832003.16 the non-zeros were: 10
And rss value of : 9.97802116168e+14

For l1 value of : 3311312564.69 the non-zeros were: 8
And rss value of : 1.03381440214e+15

For l1 value of : 3478793126.21 the non-zeros were: 7
And rss value of : 1.04967257541e+15

For l1 value of : 3646273687.74 the non-zeros were: 6
And rss value of : 1.06793032282e+15

For l1 value of : 3813754249.27 the non-zeros were: 6
And rss value of : 1.08404612419e+15

For l1 value of : 3981234810.8 the non-zeros were: 6
And rss value of : 1.10253230983e+15

For l1 value of : 4148715372.33 the non-zeros were: 6
And rss value of : 1.12549180194e+15

For l1 value of : 4316195933.85 the non-zeros were: 6
And rss value of : 1.15109771081e+15

For l1 value of : 4483676495.38 the non-zeros were: 6
And rss value of : 1.18043371645e+15

For l1 value of : 4651157056.91 the non-zeros were: 6
And rss value of : 1.2106

In [50]:
best_model_with_7.get('coefficients').print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 223853.354326 |  None  |
|     bedrooms     |  None | 550.470071563 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15716.6303041 |  None  |
|   sqft_living    |  None | 32.2718853021 |  None  |
| sqft_living_sqrt |  None | 682.560585124 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2849.30704821 |  None  |
|    sqft_above    |  None | 29.8102546932 |  None  |
|  sqft_basement   |  None |

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

1. For l1 value of : 3478793126.21 the non-zeros were: 7
And rss value of : 1.04967257541e+15 <br>
2. intercept, bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above